<a href="https://colab.research.google.com/github/Yazeedx0/Security/blob/main/Fine_Tuning_JAIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **First Fine Tuning**

In [ ]:
!pip install torch
!pip install transformers
!pip install datasets

**Download The Jais-Model**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from datasets import load_dataset

dataset = load_dataset('csv', data_files='/content/QA-Fulldata.csv')


In [24]:
from datasets import load_dataset
import pandas as pd

df = pd.read_csv('/content/QA-Fulldata.csv')
df['text'] = df['Question'] + " " + df['Answer']
df = df.dropna(subset=['text'])
df = df[df['text'].str.strip() != ""]
df.to_csv('/content/QA-Fulldata.csv', index=False)

dataset = load_dataset('csv', data_files={'train': '/content/QA-Fulldata.csv'})

Generating train split: 0 examples [00:00, ? examples/s]

In [25]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "/content/results/final_model"
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [26]:
def tokenize_function(examples):
    encoding = tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=64
    )
    encoding["labels"] = encoding["input_ids"].copy()
    return encoding

train_dataset = dataset["train"].map(tokenize_function, batched=True)
columns_to_remove = ["text", "context", "question", "answer"]
existing_columns = [col for col in columns_to_remove if col in train_dataset.column_names]
train_dataset = train_dataset.remove_columns(existing_columns)
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/58381 [00:00<?, ? examples/s]

In [27]:
print (train_dataset[0])

{'input_ids': tensor([  445,   823,  2738,  7713, 19200,   922, 13094,   357,  7976, 38959,
         1598,   603,  2329, 56888,  1128,  6047, 32299, 35273,  1042,  4117,
         1574, 13733,    14,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 'labels': tensor([  445,   823,  2738,  7713, 19200,   922, 13094,   357,  7976, 38959,
         1598,   603,  2329, 56888,  1128,  6047, 32299, 35273,  1042,  4117,
         1574, 13733,    14,     0,     0,     0,     0,     0,     0,     0,


In [30]:
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
import torch

model_name = "/content/results/final_model"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    ignore_mismatched_sizes=True,
    tie_word_embeddings=False
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

training_args = TrainingArguments(
    output_dir="./results2",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    save_total_limit=2,
    fp16=True,
    logging_dir="./logs",
    logging_steps=500,
    learning_rate=2e-5,
    weight_decay=0.01,
    warmup_steps=500,
    report_to="tensorboard"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset
)

trainer.train()


model.save_pretrained(
    "./results2/final_model",
    safe_serialization=False
)

tokenizer.save_pretrained("./results2/final_model")


Step,Training Loss
500,1.226000
1000,0.940000
1500,0.859200
2000,0.794100
2500,0.763100
3000,0.719100
3500,0.686100
4000,0.659100
4500,0.633700
5000,0.611300


('./results2/final_model/tokenizer_config.json',
 './results2/final_model/special_tokens_map.json',
 './results2/final_model/tokenizer.json')

In [ ]:
from torch.nn import Parameter

model.lm_head.weight = Parameter(model.lm_head.weight.clone())


In [ ]:
model.save_pretrained(
    "./results2/final_model2",
    safe_serialization=False
)

tokenizer.save_pretrained("./results2/final_model2")

('./results/final_model/tokenizer_config.json',
 './results/final_model/special_tokens_map.json',
 './results/final_model/tokenizer.json')

In [ ]:
!pip install gradio

In [57]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import gradio as gr
from transformers import AutoTokenizer, AutoModelForCausalLM

model_path = "/content/results2/final_model"


prompt_ar = "### Instruction: اسمك \"جيس\" وسميت على اسم جبل جيس، أعلى جبل في الإمارات. تم بناؤك بواسطة Inception في الإمارات. أنت مساعد مفيد ومحترم وصادق، وقد تم تصميمك خصيصًا للإجابة على أسئلة الطلبة في الجامعة الهاشمية. عندما يُطرح عليك سؤال، يجب أن تجيب عليه بشكل دقيق وصحيح بناءً على أفضل المعلومات المتوفرة لديك. احرص دائمًا على تقديم إجابة واضحة ومبنية على حقائق صحيحة، مع الحفاظ على الأمان والإحترام في الإجابة. إذا كنت لا تعرف الإجابة بدقة، يجب أن توضح ذلك بأدب. أكمل المحادثة بين [|Human|] و[|AI|] :\n### Input:[|Human|] {Question}\n[|AI|]\n### Response :"

device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True, device_map="auto")


def get_response(text, tokenizer=tokenizer, model=model):
    input_ids = tokenizer(text, return_tensors="pt").input_ids
    inputs = input_ids.to(device)
    input_len = inputs.shape[-1]
    generate_ids = model.generate(
        inputs,
        top_p=0.9,
        temperature=0.3,
        max_length=2048,
        min_length=input_len + 4,
        repetition_penalty=1.2,
        do_sample=True,
    )
    response = tokenizer.batch_decode(
        generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True
    )[0]
    response = response.split("### Response :")[-1]
    return response

def generate_gradio_response(question):
    text = prompt_ar.format_map({'Question': question})
    return get_response(text)

interface = gr.Interface(
    fn=generate_gradio_response,
    inputs=[gr.Textbox(label="أدخل سؤالك هنا")],
    outputs="text",
    title="مساعد الجامعة الهاشمية",
    description="أدخل سؤالك، وسأقوم بالإجابة عليه باستخدام النموذج المدرب."
)

interface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e5d0e7503af81b610c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [58]:
import shutil

source_path = "/content/results2"

destination_path = "/content/drive/My Drive/results2"

shutil.copytree(source_path, destination_path)

print("Folder copied successfully to Google Drive.")


Folder copied successfully to Google Drive.
